In [26]:
# import libraries
import pandas as pd
import numpy as np
import os


In [29]:
ords_prods_merge = pd.read_pickle(r'C:\CareerFoundary\Achivement 4.7\data\Prepared Data\ords_prods_merge.pkl')

In [5]:
ords_prods_merge.head()

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,_merge,product_name,aisle_id,department_id,prices,price_label,busiest_day,slowest_day,busiest_period_of_day
0,2539329,1,prior,1,2,8,0.0,196.0,1.0,0.0,both,Soda,77.0,7.0,9.0,Mid,Regular Day,Regular Day,Average orders
1,2539329,1,prior,1,2,8,0.0,14084.0,2.0,0.0,both,Organic Unsweetened Vanilla Almond Milk,91.0,16.0,12.5,Mid,Regular Day,Regular Day,Average orders
2,2539329,1,prior,1,2,8,0.0,12427.0,3.0,0.0,both,Original Beef Jerky,23.0,19.0,4.4,Low,Regular Day,Regular Day,Average orders
3,2539329,1,prior,1,2,8,0.0,26088.0,4.0,0.0,both,Aged White Cheddar Popcorn,23.0,19.0,4.7,Low,Regular Day,Regular Day,Average orders
4,2539329,1,prior,1,2,8,0.0,26405.0,5.0,0.0,both,XL Pick-A-Size Paper Towel Rolls,54.0,17.0,1.0,Low,Regular Day,Regular Day,Average orders


In [7]:
# Group the dataframe by 'department_id' and calculate the mean of 'order_number'
order_number_mean_by_dept = ords_prods_merge.groupby('department_id')['order_number'].mean()

# Print the results to see the aggregated mean
print(order_number_mean_by_dept)


department_id
1.0     15.457687
2.0     17.277920
3.0     17.179756
4.0     17.811403
5.0     15.213779
6.0     16.439806
7.0     17.225773
8.0     15.340520
9.0     15.895474
10.0    20.197148
11.0    16.170828
12.0    15.887622
13.0    16.583304
14.0    16.757377
15.0    16.165037
16.0    17.663250
17.0    15.694469
18.0    19.310397
19.0    17.177343
20.0    16.473447
21.0    22.902379
Name: order_number, dtype: float64


In [9]:
# Calculate the mean of the 'order_number' column for the entire dataframe
mean_order_number = ords_prods_merge['order_number'].mean()

# Print the result
print(mean_order_number)


17.138388999600213


In [11]:
# Calculate the mean of the 'order_number' column for the entire dataframe
mean_order_number = ords_prods_merge['order_number'].mean()

# Print the result
print(mean_order_number)


17.138388999600213


In [13]:
# Step 1: Calculate the number of orders for each user using 'transform()'
ords_prods_merge['order_count'] = ords_prods_merge.groupby('user_id')['order_number'].transform('max')

In [15]:
ords_prods_merge.loc[ords_prods_merge['order_count'] > 5, 'loyalty_flag'] = 'Loyal'
ords_prods_merge.loc[ords_prods_merge['order_count'] <= 5, 'loyalty_flag'] = 'Non-Loyal'

In [17]:
print(ords_prods_merge[['user_id', 'order_count', 'loyalty_flag']].head())

   user_id  order_count loyalty_flag
0        1           11        Loyal
1        1           11        Loyal
2        1           11        Loyal
3        1           11        Loyal
4        1           11        Loyal


In [19]:
loyalty_spending_stats = ords_prods_merge.groupby('loyalty_flag')['prices'].describe()

In [21]:
print(loyalty_spending_stats)

                   count       mean         std  min  25%  50%   75%      max
loyalty_flag                                                                 
Loyal         30992664.0  11.934485  491.121606  1.0  4.2  7.4  11.3  99999.0
Non-Loyal      1441548.0  12.984293  580.394476  1.0  4.2  7.4  11.3  99999.0


In [23]:
#Group by 'user_id' and calculate the mean of 'prices'
user_spending = ords_prods_merge.groupby('user_id')['prices'].mean()
#Create a new column for spending flag based on the mean price
ords_prods_merge['spending_flag'] = ords_prods_merge['user_id'].map(lambda x: 'High spender' if user_spending[x] >= 10 else 'Low spender')
#Check the result
print(ords_prods_merge[['user_id', 'spending_flag']].head())


   user_id spending_flag
0        1   Low spender
1        1   Low spender
2        1   Low spender
3        1   Low spender
4        1   Low spender


In [25]:
#Calculate the mean price of products purchased by each user
user_avg_spending = ords_prods_merge.groupby('user_id')['prices'].mean()

#Create a new column "spending_flag" and assign values based on conditions
ords_prods_merge['spending_flag'] = ords_prods_merge['user_id'].map(
    lambda x: 'High spender' if user_avg_spending[x] >= 10 else 'Low spender'
)

#Print value counts to check the distribution of spenders
print(ords_prods_merge['spending_flag'].value_counts())

# Display a sample of the new column
print(ords_prods_merge[['user_id', 'spending_flag']].head())


spending_flag
Low spender     32001528
High spender      640922
Name: count, dtype: int64
   user_id spending_flag
0        1   Low spender
1        1   Low spender
2        1   Low spender
3        1   Low spender
4        1   Low spender


In [31]:
#Calculate the median days_since_prior_order per user
user_median_order_frequency = ords_prods_merge.groupby('user_id')['days_since_prior_order'].median()

#Create a new column "order_frequency_flag" based on conditions
ords_prods_merge['order_frequency_flag'] = ords_prods_merge['user_id'].map(
    lambda x: 'Non-frequent customer' if user_median_order_frequency[x] > 20 
    else ('Regular customer' if 10 < user_median_order_frequency[x] <= 20 
          else 'Frequent customer')
)

# Print value counts to check the distribution
print(ords_prods_merge['order_frequency_flag'].value_counts())
print(ords_prods_merge[['user_id', 'order_frequency_flag']].head())

order_frequency_flag
Frequent customer        22878061
Regular customer          7011736
Non-frequent customer     2752653
Name: count, dtype: int64
   user_id order_frequency_flag
0        1     Regular customer
1        1     Regular customer
2        1     Regular customer
3        1     Regular customer
4        1     Regular customer


In [33]:
file_path = r'C:\CareerFoundary\Achivement 4.7\data\Prepared Data\ords_prods_merge.pkl'
ords_prods_merge.to_pickle(file_path)
print("Dataframe successfully saved as a pickle file.")

Dataframe successfully saved as a pickle file.
